# Standardization steps for the death data integration process. 


### 1 Normalization of individual Variables

In [ ]:
%%bigquery  --project $project_dev
-- defined functions  to be used in all Tokens
-- We want to return Null when the input is ''
--We want everything to be upper case


CREATE OR REPLACE FUNCTION Death_Data_Integration.IsValid_DATE(input ANY Type)
--Return "" if invalid DOB else return the DOB
-- A Valid DOB is a date in the format YYYYMMDD
-- A 4-digit year from 0000 to 2022
-- Followed by a 2 digit month from 01 to 12 (padded with leading zeros if necessary)
-- Followed by a 2 digit day from 01 to 31 (padded with leading zeros if necessary)
-- TODO: remove date 1901-01-01
RETURNS STRING

  AS (

    IF(
      (
        cast(
              LEFT
                (
                  regexp_replace(TRIM(cast(input as string)), '[^0-9]', ''),4
                ) as INT64 )
                <1850 -- remove dates before 1850
         OR 
         
         cast(
              LEFT
                (
                  regexp_replace(TRIM(cast(input as string)), '[^0-9]', ''),4
                ) as INT64 )
                >(SELECT EXTRACT(YEAR FROM CURRENT_DATETIME() )) -- remove dates > than the current year
      )
         ,'',input 
      ) 
  );


CREATE OR REPLACE FUNCTION Death_Data_Integration.IsValid_SSN(input STRING)
--Return "" if invalid SSN else return the SSN
-- An invalid SSN is one that the SSA never assigned. In case you’re wondering, a valid SSN will never look like this:

-- The first three digits as “000,” “666,” or in the 900 series.
-- The second group that consists of two digits as “00.”
-- The third group consisting of four digits as “0000.”

RETURNS STRING
  AS (
      REGEXP_replace(
        input,
        '^((666|000|[9][0-9][0-9])\\d{2}\\d{4})|(\\d{3}00\\d{4})|(\\d{3}\\d{2}0000)0{9}|1{9}|2{9}|3{9}|4{9}|5{9}|6{9}|7{9}|8{9}|9{9}|219099999|078051120|123456789|012345678|001010001|090909090$',
        ""
        )
  );



CREATE OR REPLACE FUNCTION Death_Data_Integration.Normalize_SSN(input ANY Type)
  RETURNS STRING
  AS (
    NULLIF
    (
      Death_Data_Integration.IsValid_SSN(
                  UPPER(
                    regexp_replace(TRIM(input), '[^0-9]', '')
                      )
                    ),
     '')
        
     );

CREATE OR REPLACE  FUNCTION Death_Data_Integration.Normalize_String(input STRING)
  RETURNS STRING
  AS (
      NULLIF
      (
        UPPER
        (
          regexp_replace(TRIM(input), '[^a-zA-Z]', '')),'')
     );

CREATE OR REPLACE  FUNCTION Death_Data_Integration.Normalize_Middle_Name(input STRING)
  RETURNS STRING
  AS (
      NULLIF
      (
        LEFT (  -- trucating string  
          UPPER
        (
          regexp_replace(TRIM(input), '[^a-zA-Z]', '')
          )
            ,15),
        '')
     );

CREATE OR REPLACE  FUNCTION Death_Data_Integration.Normalize_First_Name(input STRING)
  RETURNS STRING
  AS (
      NULLIF
      (
        LEFT (  -- trucating string  
          UPPER
        (
          regexp_replace(TRIM(input), '[^a-zA-Z]', '')
          )
            ,15),
        '')
     );

CREATE OR REPLACE  FUNCTION Death_Data_Integration.Normalize_Last_Name(input STRING)
  RETURNS STRING
  AS (
      NULLIF
      (
        LEFT (
          UPPER
        (
          regexp_replace(TRIM(input), '[^a-zA-Z]', '')
          )
            ,20),
        '')
     );


CREATE OR REPLACE  FUNCTION Death_Data_Integration.Normalize_DATE(input ANY Type)
  RETURNS STRING
  AS (
    NULLIF
    (
      Death_Data_Integration.IsValid_DATE(
      UPPER
      (
        LEFT
        (
          regexp_replace(TRIM(cast(input as string)), '[^0-9]', ''),8)
      )
    )
      ,'')
     );

-- 1st initial of first name
CREATE OR REPLACE  FUNCTION Death_Data_Integration.Normalize_Extract_First_N_Characters(input STRING, characters INT64)
  RETURNS STRING
  AS (
      NULLIF
      (
        UPPER
        (
          LEFT
          (
            regexp_replace
            (
                TRIM(input), '[^a-zA-Z]', ''), characters)),'')
     );

-- SOUNDEX
CREATE OR REPLACE  FUNCTION Death_Data_Integration.Normalize_Soundex(input STRING)
  RETURNS STRING
  AS (
    NULLIF
    (
      UPPER
      (
        SOUNDEX(
          regexp_replace(TRIM(input), '[^a-zA-Z]', ''))),'')
     );

-- extract Year
CREATE OR REPLACE  FUNCTION Death_Data_Integration.Normalize_Extract_Year(input ANY Type)
  RETURNS STRING
  AS (
    NULLIF
    (
      Death_Data_Integration.IsValid_DATE(
      UPPER
      (
        LEFT
        (
          regexp_replace(TRIM(cast(input as string)), '[^0-9]', ''),4)
          )
      )
          ,'')
     );

-- extract last 4 digits of SSN
CREATE OR REPLACE  FUNCTION Death_Data_Integration.Normalize_Extract_Last_4_Digits_SSN(input ANY Type)
  RETURNS STRING
  AS (
    NULLIF
    (
      RIGHT
                  (
      Death_Data_Integration.IsValid_SSN
      (
            UPPER
                (
                  
                    regexp_replace(TRIM(cast(input as string)), '[^0-9]', '')

                    )
                    
                    )
             ,4),
            '')
           
     );
     




### 2 Token Creation

In [ ]:
%%bigquery --project $project_dev

-- CONCAT: The function returns NULL if any input argument is NULL.


-- Token_1  ssn+last_name+middle_name+first_name_name+birth_date
CREATE OR REPLACE FUNCTION Death_Data_Integration.Token_1(
        ssn ANY Type,
        last_name STRING,
        middle_name STRING,
        first_name STRING,
        birth_date ANY Type
    ) RETURNS STRING AS (
        CONCAT(
            Death_Data_Integration.Normalize_SSN(ssn),
            Death_Data_Integration.Normalize_Last_Name(last_name),
            Death_Data_Integration.Normalize_Middle_Name(middle_name),
            Death_Data_Integration.Normalize_First_Name(first_name),
            Death_Data_Integration.Normalize_DATE(birth_date)
        )
    ) OPTIONS (
        description = "Token_1:  ssn+last_name+middle_name+first_name_name+birth_date"
    );
    -- Token_2	ssn +	first20(last_name)	+ first15(first_name) + 	birth_date
CREATE OR REPLACE FUNCTION Death_Data_Integration.Token_2(
        ssn ANY Type,
        last_name STRING,
        first_name STRING,
        birth_date ANY Type
    ) RETURNS STRING AS (
        CONCAT(
            Death_Data_Integration.Normalize_SSN(ssn),
            Death_Data_Integration.Normalize_Last_Name(last_name),
            Death_Data_Integration.Normalize_First_Name(first_name),
            Death_Data_Integration.Normalize_DATE(birth_date)
        )
    ) OPTIONS (
        description = "Token_2:  ssn+last_name+first_name_name+birth_date"
    );

-- Token 3: SSN+DOB
CREATE OR REPLACE FUNCTION Death_Data_Integration.Token_3(ssn ANY Type, birth_date ANY Type) RETURNS STRING AS (
        CONCAT(
            Death_Data_Integration.Normalize_SSN(ssn),
            Death_Data_Integration.Normalize_DATE(birth_date)
        )
    ) OPTIONS (description = "Token 3: SSN +  DOB");



-- Token_4  exact birth year + first name + last name  
CREATE OR REPLACE FUNCTION Death_Data_Integration.Token_4(
        ssn ANY Type,
        last_name STRING,
        birth_date ANY Type,
        first_name STRING
    ) RETURNS STRING AS (
        CONCAT(
            Death_Data_Integration.Normalize_SSN(ssn),
            Death_Data_Integration.Normalize_Extract_Year(birth_date),
            Death_Data_Integration.Normalize_Last_Name(last_name),
            Death_Data_Integration.Normalize_First_Name(first_name)
        )
    ) OPTIONS (
        description = "Token_4:  ssn + exact birth year + last name + first name"
    );

-- Token_5  ssn+last_name+middle_name+first_name
CREATE OR REPLACE FUNCTION Death_Data_Integration.Token_5(
        ssn ANY Type,
        last_name STRING,
        middle_name STRING,
        first_name STRING
    ) RETURNS STRING AS (
        CONCAT(
            Death_Data_Integration.Normalize_SSN(ssn),
            Death_Data_Integration.Normalize_Last_Name(last_name),
            Death_Data_Integration.Normalize_Middle_Name(middle_name),
            Death_Data_Integration.Normalize_First_Name(first_name)
        )
    ) OPTIONS (
        description = "Token_5:  ssn+last_name+middle_name+first_name"
    );



-- Token_6	SSN
CREATE OR REPLACE FUNCTION Death_Data_Integration.Token_6(ssn ANY Type) RETURNS STRING AS (
        Death_Data_Integration.Normalize_SSN(ssn)
    ) OPTIONS (description = "Token_6: only ssn");



-- Token_7	last4(ssn)	first20(last_name) middle_name	first15(first_name)	+ birth_date
CREATE OR REPLACE FUNCTION Death_Data_Integration.Token_7(
        ssn ANY Type,
        last_name STRING,
        middle_name STRING,
        first_name STRING,
        birth_date ANY Type
    ) RETURNS STRING AS (
        CONCAT(
            Death_Data_Integration.Normalize_Extract_Last_4_Digits_SSN(ssn),
            Death_Data_Integration.Normalize_Last_Name(last_name),
            Death_Data_Integration.Normalize_Middle_Name(middle_name),
            Death_Data_Integration.Normalize_First_Name(first_name),
            Death_Data_Integration.Normalize_DATE(birth_date)
        )
    ) OPTIONS (
        description = "Token_7:  last4(ssn)	+ first20(last_name)+ middle_name+ 	first15(first_name)	+ birth_date"
    );
    
-- Token_8	SSN (last 4) + DoB
CREATE OR REPLACE FUNCTION Death_Data_Integration.Token_8(ssn ANY Type, birth_date ANY Type) RETURNS STRING AS (
        CONCAT(
            Death_Data_Integration.Normalize_Extract_Last_4_Digits_SSN(ssn),
            Death_Data_Integration.Normalize_DATE(birth_date)
        )
    ) OPTIONS (description = "Token_8:  SSN (last 4) + DoB");


-- Token 9: last_name	+middle_name+	first_name_name	+birth_date
CREATE OR REPLACE FUNCTION Death_Data_Integration.Token_9(
        last_name STRING,
        middle_name STRING,
        first_name STRING,
        birth_date ANY Type
    ) RETURNS STRING AS (
        CONCAT(
            Death_Data_Integration.Normalize_Last_Name(last_name),
            Death_Data_Integration.Normalize_Middle_Name(middle_name),
            Death_Data_Integration.Normalize_First_Name(first_name),
            Death_Data_Integration.Normalize_DATE(birth_date)
        )
    ) OPTIONS (
        description = "Token 9:first20(last_name) + first15(middle_name) + first15(first_name) + birth_date"
    );
-- Token 10: last_name +	first_name	+ birth_date
CREATE OR REPLACE FUNCTION Death_Data_Integration.Token_10(
        last_name STRING,
        middle_name STRING,
        first_name STRING,
        birth_date ANY Type
    ) RETURNS STRING AS (
        CONCAT(
            Death_Data_Integration.Normalize_Last_Name(last_name),
            Death_Data_Integration.Normalize_Middle_Name(middle_name),
            Death_Data_Integration.Normalize_First_Name(first_name),
            Death_Data_Integration.Normalize_Extract_Year(birth_date)
        )
    ) OPTIONS (
        description = "Token 10: first20(last_name)  + first15(first_name) + birth_date"
    );
-- Token 11: last name + first name + DOB
CREATE OR REPLACE FUNCTION Death_Data_Integration.Token_11(
        last_name STRING,
        first_name STRING,
        birth_date ANY Type
    ) RETURNS STRING AS (
        CONCAT(
            Death_Data_Integration.Normalize_Last_Name(last_name),
            Death_Data_Integration.Normalize_First_Name(first_name),
            Death_Data_Integration.Normalize_DATE(birth_date)
        )
    ) OPTIONS (
        description = "Token 11 last name + first name + DOB"
    );

-- Token 12 last name + 1st initial of middle name + first name 
CREATE OR REPLACE FUNCTION Death_Data_Integration.Token_12(
        last_name STRING,
        middle_name STRING,
        first_name STRING
    ) RETURNS STRING AS (
        CONCAT(
            Death_Data_Integration.Normalize_Last_Name(last_name),
            Death_Data_Integration.Normalize_Extract_First_N_Characters(middle_name, 1),
            Death_Data_Integration.Normalize_First_Name(first_name)
        )
    ) OPTIONS (
        description = "Token 12: last name + 1st initial of middle name + first name"
    );    
-- Token 13: last name + 1st 3 characters of first name + DOB
CREATE OR REPLACE FUNCTION Death_Data_Integration.Token_13(
        last_name STRING,
        first_name STRING,
        birth_date ANY Type
    ) RETURNS STRING AS (
        CONCAT(
            Death_Data_Integration.Normalize_Last_Name(last_name),
            Death_Data_Integration.Normalize_Extract_First_N_Characters(first_name, 3),
            Death_Data_Integration.Normalize_DATE(birth_date)
        )
    ) OPTIONS (
        description = "Token 13: last name + 1st 3 characters of first name + DOB"
    );



-- Token 14: last name + 1st initial of first name + DOB
CREATE OR REPLACE FUNCTION Death_Data_Integration.Token_14(
        last_name STRING,
        first_name STRING,
        birth_date ANY Type
    ) RETURNS STRING AS (
        CONCAT(
            Death_Data_Integration.Normalize_Last_Name(last_name),
            Death_Data_Integration.Normalize_Extract_First_N_Characters(first_name, 1),
            Death_Data_Integration.Normalize_DATE(birth_date)
        )
    ) OPTIONS (
        description = " Token 14: Creates a token as a combination of last name + 1st initial of first name + DOB"
    );

-- Token 15: last name (soundex) + middle_name  (soundex) + first name (soundex) + birth_date
CREATE OR REPLACE FUNCTION Death_Data_Integration.Token_15(
        last_name STRING,
        middle_name STRING,
        first_name STRING,
        birth_date ANY Type
    ) RETURNS STRING AS (
      
        CONCAT(
            Death_Data_Integration.Normalize_Soundex(
                Death_Data_Integration.Normalize_Last_Name(last_name)
            ),
            Death_Data_Integration.Normalize_Soundex(
            Death_Data_Integration.Normalize_Middle_Name(middle_name)
            ),
            Death_Data_Integration.Normalize_Soundex(
                Death_Data_Integration.Normalize_First_Name(first_name)
            ),
            Death_Data_Integration.Normalize_DATE(birth_date)
        )
    ) OPTIONS (
        description = "Token 15: last name (soundex) + middle_name  (soundex) + first name (soundex) + birth_date"
    );

-- Token 16: last name (soundex) + middle_name  (soundex) + first name (soundex) + birth_date
CREATE OR REPLACE FUNCTION Death_Data_Integration.Token_16(
        last_name STRING,
        middle_name STRING,
        first_name STRING,
        birth_date ANY Type
    ) RETURNS STRING AS (
      
        CONCAT(
            Death_Data_Integration.Normalize_Soundex(
                Death_Data_Integration.Normalize_Last_Name(last_name)
            ),
            Death_Data_Integration.Normalize_Soundex(
            Death_Data_Integration.Normalize_Middle_Name(middle_name)
            ),
            Death_Data_Integration.Normalize_Soundex(
                Death_Data_Integration.Normalize_First_Name(first_name)
            ),
            Death_Data_Integration.Normalize_Extract_Year(birth_date)
        )
    ) OPTIONS (
        description = "Token 16: last name (soundex) + middle_name  (soundex) + first name (soundex) + birth_date"
    );




-- Token 17: last name (soundex) + first name (soundex) + DOB
CREATE OR REPLACE FUNCTION Death_Data_Integration.Token_17(
        last_name STRING,
        first_name STRING,
        birth_date ANY Type
    ) RETURNS STRING AS (
        CONCAT(
            Death_Data_Integration.Normalize_Soundex(
                Death_Data_Integration.Normalize_Last_Name(last_name)
            ),
            Death_Data_Integration.Normalize_Soundex(
                Death_Data_Integration.Normalize_First_Name(first_name)
            ),
            Death_Data_Integration.Normalize_DATE(birth_date)
        )
    ) OPTIONS (
        description = "Token 17: last name (soundex) + first name (soundex) + DOB"
    );

-- Token_18	last_name  
CREATE OR REPLACE FUNCTION Death_Data_Integration.Token_18(last_name STRING) RETURNS STRING AS (
        Death_Data_Integration.Normalize_Last_Name(last_name)
    ) OPTIONS (description = "Token_18:  last_name");

-- Token_19	first_name
CREATE OR REPLACE FUNCTION Death_Data_Integration.Token_19(first_name STRING) RETURNS STRING AS (
        Death_Data_Integration.Normalize_First_Name(first_name)
    ) OPTIONS (description = "Token_19:  first_name");

-- Token_20	DoB
CREATE OR REPLACE FUNCTION Death_Data_Integration.Token_20(birth_date ANY Type) RETURNS STRING AS (
        Death_Data_Integration.Normalize_DATE(birth_date)
    ) OPTIONS (description = "Token_20:  DoB");






